# Lectura de documentos


En primer lugar, importamos las librerías necesarias para la ejecución del código.

In [1]:
!pip install pymupdf
!pip install pdfplumber

import os
import re
import pdfplumber
from tabulate import tabulate
import warnings

warnings.filterwarnings("ignore")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 98.7 MB/s eta 0:00:00


En segundo lugar, se realiza la lectura de los documentos utilizados diferenciando entre texto libre y tablas.

a) *clean_text_from_table_rows*: Evita duplicidades entre el texto plano y las tablas extraídas, eliminando del texto aquellas filas que también están en las tablas.

b) *extract_text_and_tables_per_page*: Extrae el texto y las tablas de cada página del PDF y los fusiona inteligentemente.
Devuelve una lista de strings por página.

c) *read_all_pdfs_page_by_page*: Recorre todos los archivos PDF de una carpeta, y para cada uno usa extract_text_and_tables_per_page.
→ Output: [[página1, página2, ...], [página1, página2, ...], ...]

In [2]:
def clean_text_from_table_rows(text, tables):
    """
    Elimina del texto libre cualquier fila de las tablas que se encuentre literal en el texto.
    """
    for table in tables:
        for row in table:
            row_str = " ".join(cell.strip() if cell else "" for cell in row).strip()
            if row_str in text:
                text = text.replace(row_str, "")
    return text


def extract_text_and_tables_per_page(pdf_path):
    """
    Extrae el texto y las tablas de cada página, eliminando duplicidad.
    Devuelve lista de strings (uno por cada página del documento).
    """
    page_texts = []

    with pdfplumber.open(pdf_path) as pdf:

        for page in pdf.pages:
            text = page.extract_text() or ""
            tables = page.extract_tables()

            # Limpieza del texto para evitar duplicación con tablas
            text = clean_text_from_table_rows(text, tables)

            # Normalización de espacios
            text = re.sub(r"\s+", " ", text).strip()

            # Procesar tablas
            tables_text = ""
            for table in tables:
                if table:  # Verifica que la tabla no esté vacía
                    tables_text += tabulate(table, tablefmt="grid") + "\n\n"

            # Combinar resultados
            if text.strip() and tables_text.strip():
                full_content = f"{text.strip()}\n\nTABLAS:\n{tables_text.strip()}"
            elif tables_text.strip():
                full_content = f"TABLAS:\n{tables_text.strip()}"
            else:
                full_content = text.strip()

            page_texts.append(full_content)

    return page_texts


def read_all_pdfs_page_by_page(folder_path):
    """
    Lee todos los PDFs de una carpeta y devuelve una lista cuya longitud es el número de documentos.
    Cada elemento de la lista es una lista de strings (uno por cada página del documento).
    """
    all_docs = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            print(f"Procesando: {filename}")
            doc_pages = extract_text_and_tables_per_page(pdf_path)
            all_docs.append(doc_pages)

    return all_docs

In [3]:
# Ejemplo de uso

# Ruta a los documentos
from google.colab import drive
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/proyecto_pdfs"
textos = read_all_pdfs_page_by_page(folder_path)

print(f"Número de documentos: {len(textos)}")
print(f"Número de páginas del primer documento: {len(textos[0])}")
print(f"\nContenido de la primera página del primer PDF:\n{textos[0][0]}")

Mounted at /content/drive
Procesando: Cómo_modificar_una_declaración_ya_presentada.pdf
Procesando: ManualRenta2024Tomo1_es_es.pdf
Procesando: Manual_práctico_de_Patrimonio_2024..pdf
Procesando: BOE-064_Impuesto_sobre_la_Renta_de_las_Personas_Fisicas.pdf


Procesando: ManualRenta2024Tomo2_es_es.pdf
Procesando: Manual_práctico_de_Sociedades_2024..pdf
Procesando: Manual_práctico_IVA_2024..pdf
Procesando: BOE-A-2007-6820-consolidado.pdf
Procesando: Preguntas_frecuentes.pdf
Procesando: BOE-A-2025-5049-consolidado.pdf
Procesando: taxdown-guia-deducciones-fiscales-2025.pdf


Procesando: RentaWEB2023.pdf
Procesando: BOE-A-2006-20764-consolidado.pdf
Número de documentos: 13
Número de páginas del primer documento: 28

Contenido de la primera página del primer PDF:
Cómo modificar una declaración ya presentada Cómo modificar una declaración ya presentada Índice • Declaraciones del ejercicio 2024 • Introducción • Ejemplo de una autoliquidación rectificativa con un mayor importe a ingresar • Ejemplo de una autoliquidación rectificativa con un menor importe a ingresar • Ejemplo de una autoliquidación rectificativa a devolver • Cambios de opción • Declaraciones del ejercicio 2023 • Introducción • Primer paso: modificar datos • Segundo paso: modificar la declaración ya presentada • Tercer paso: presentar declaración • Declaraciones del ejercicio 2022 y anteriores • Introducción • Primer paso: modificar datos • Segundo paso: modificar la declaración ya presentada • Tercer paso: presentar declaración • Glosario de abreviaturas 26/05/2025 - Cómo modificar una declaraci

# Limpieza de texto

A continuación, se lleva a cabo la limpieza del texto con el objetivo de reducir ruido y redundancia, obtener uniformidad para embeddings y eliminar caracteres raros que puedan afectar a la vectorización.

a) *limpiar_pagina*:
Aplica una limpieza línea por línea del texto:

Elimina pies de página, cabeceras inútiles, líneas vacías o con símbolos, etc.

Añade las tablas al final del texto limpio.

b) *limpiar_documentos*:
Aplica limpiar_pagina a cada página de cada documento.

In [4]:
def limpiar_pagina(texto_pagina, tabla_pagina=""):
    """
    Limpia texto de la página línea a línea:
    - Quita líneas vacías
    - Quita pies de página estilo "- 644 -"
    - Quita numeración de página simple ("12" o "Página 12")
    - Quita cabeceras cortas en mayúsculas (<=6 palabras)
    - Quita líneas con solo símbolos/dígitos/guiones bajos
    - Quita líneas de separación tipo "-----" o "_____"
    - Normaliza espacios
    Deja las tablas intactas y las añade al final.
    """
    # Separa el texto en líneas individuales
    lineas = texto_pagina.split("\n")
    nuevas_lineas = []

    # Procesa cada línea para decidir si se mantiene o se descarta
    for linea in lineas:
        linea = linea.strip()  # Elimina espacios al inicio y final
        if not linea:
            continue  # Salta líneas vacías

        # Ignora pies de página tipo "- 644 -"
        if re.match(r"^-+\s*\d+\s*-+$", linea):
            continue

        # Ignora numeración de página simple ("12" o "Página 12")
        if re.match(r"^(Página\s*)?\d+$", linea, re.IGNORECASE):
            continue

        # Ignora cabeceras cortas en mayúsculas (hasta 6 palabras)
        if linea.isupper() and len(linea.split()) <= 6:
            continue

        # Ignora líneas con solo símbolos, dígitos o guiones bajos (3 o más caracteres)
        if re.match(r"^[\W\d_]{3,}$", linea):
            continue

        # Ignora líneas de separación como "-----" o "_____"
        if re.match(r"^[\-\_]{3,}$", linea):
            continue

        # Reemplaza múltiples espacios consecutivos por uno solo
        linea = re.sub(r"\s{2,}", " ", linea)

        # Añade la línea limpia a la lista final
        nuevas_lineas.append(linea)

    # Une las líneas filtradas para formar el texto limpio de la página
    texto_limpio = "\n".join(nuevas_lineas)

    # Si hay tablas, las añade al final del texto limpio (con dos saltos de línea antes)
    if tabla_pagina.strip():
        texto_limpio += "\n\n" + tabla_pagina.strip()

    return texto_limpio


def limpiar_documentos(textos):
    """
    Recibe lista de documentos (lista de páginas con texto + tablas),
    limpia todas las páginas sin eliminar ninguna.
    Devuelve documentos limpios.
    """
    documentos_limpios = []

    # Recorre cada documento (lista de páginas)
    for documento in textos:
        paginas_limpias = []

        # Recorre cada página del documento
        for pagina in documento:
            # Si la página contiene tablas añadidas, las separa del texto
            if "TABLAS:\n" in pagina:
                partes = pagina.split("TABLAS:\n", 1)
                texto_sin_tablas = partes[0].strip()
                tabla = partes[1].strip()
            else:
                texto_sin_tablas = pagina
                tabla = ""

            # Limpia la página con la función definida antes, dejando tablas al final
            pagina_limpia = limpiar_pagina(texto_sin_tablas, tabla)

            # Añade la página limpia solo si tiene contenido
            if pagina_limpia.strip():
                paginas_limpias.append(pagina_limpia)

        # Añade el documento limpio (lista de páginas limpias) al resultado final
        documentos_limpios.append(paginas_limpias)

    return documentos_limpios

In [5]:
# Ejemplo de uso
textos_limpios = limpiar_documentos(textos)

print(f"Número de documentos: {len(textos_limpios)}")
print(f"Número de páginas del primer documento: {len(textos_limpios[0])}")
print(f"\nContenido de la primera página del primer PDF:\n{textos_limpios[0][0]}")

Número de documentos: 13
Número de páginas del primer documento: 28

Contenido de la primera página del primer PDF:
Cómo modificar una declaración ya presentada Cómo modificar una declaración ya presentada Índice • Declaraciones del ejercicio 2024 • Introducción • Ejemplo de una autoliquidación rectificativa con un mayor importe a ingresar • Ejemplo de una autoliquidación rectificativa con un menor importe a ingresar • Ejemplo de una autoliquidación rectificativa a devolver • Cambios de opción • Declaraciones del ejercicio 2023 • Introducción • Primer paso: modificar datos • Segundo paso: modificar la declaración ya presentada • Tercer paso: presentar declaración • Declaraciones del ejercicio 2022 y anteriores • Introducción • Primer paso: modificar datos • Segundo paso: modificar la declaración ya presentada • Tercer paso: presentar declaración • Glosario de abreviaturas 26/05/2025 - Cómo modificar una declaración ya presentada Página 1


# Dividir en chunks

a) *extraer_tablas_de_texto*:
Distingue claramente entre bloques de texto y bloques de tablas (que están en formato tabulate).
→ Devuelve: ([bloques de texto], [bloques de tablas])

b) *preparar_chunks_detectando_tablas*:
Divide los bloques de texto largos en chunks con solapamiento.

Las tablas se mantienen como bloques independientes.
→ Devuelve una lista de chunks: cada uno es un diccionario con document_id, chunk_id, text, type, y source.

In [6]:
def extraer_tablas_de_texto(texto):
    """
    Separa bloques de texto y bloques de tabla del texto completo.
    Las tablas están en formato tabulate (bordes con '+---' y columnas con '|').
    """
    lineas = texto.split("\n")
    bloques_texto = []
    bloques_tabla = []

    buffer_texto = []
    buffer_tabla = []
    dentro_de_tabla = False

    for linea in lineas:
        if re.match(r"^\+-[-+]+-\+$", linea) or (dentro_de_tabla and "|" in linea):
            # Línea de tabla (inicio o continuación)
            dentro_de_tabla = True
            buffer_tabla.append(linea)
        else:
            # Línea que no es tabla
            if dentro_de_tabla:
                # Salimos de tabla: guardar la tabla completa
                bloques_tabla.append("\n".join(buffer_tabla))
                buffer_tabla = []
                dentro_de_tabla = False
            buffer_texto.append(linea)

    # Si termina en tabla
    if buffer_tabla:
        bloques_tabla.append("\n".join(buffer_tabla))

    # Unimos el texto restante (si hay) como un solo bloque
    texto_resultante = "\n".join(buffer_texto).strip()
    bloques_texto = [texto_resultante] if texto_resultante else []

    return bloques_texto, bloques_tabla

def chunk_text(text, chunk_size=50, overlap=5):
    """
    Divide un texto largo en chunks con solapamiento.
    Devuelve una lista de strings.
    """
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + chunk_size
        chunk = words[start:end]
        chunks.append(" ".join(chunk))
        start += chunk_size - overlap  # avanzar con solapamiento

    return chunks


def preparar_chunks_detectando_tablas(textos_limpios, nombres_archivos, chunk_size=500, overlap=50):
    """
    Para cada documento:
    - Detecta bloques de texto y tablas
    - Fragmenta solo el texto
    - Deja cada tabla como un chunk único
    """
    chunks = []

    for doc_id, documento in enumerate(textos_limpios):
        nombre_archivo = nombres_archivos[doc_id]
        texto_completo = "\n".join(documento)

        bloques_texto, bloques_tabla = extraer_tablas_de_texto(texto_completo)

        chunk_id = 0
        for bloque_texto in bloques_texto:
            texto_chunks = chunk_text(bloque_texto, chunk_size=chunk_size, overlap=overlap)
            for chunk in texto_chunks:
                chunks.append({
                    "document_id": doc_id,
                    "chunk_id": f"text_{chunk_id}",
                    "text": chunk.strip(),
                    "type": "text",
                    "source": nombre_archivo
                })
                chunk_id += 1

        for table_id, tabla in enumerate(bloques_tabla):
            chunks.append({
                "document_id": doc_id,
                "chunk_id": f"table_{table_id}",
                "text": tabla.strip(),
                "type": "table",
                "source": nombre_archivo
            })

    return chunks

In [7]:
# Ejemplo de uso

nombres_archivos = [f for f in os.listdir(folder_path) if f.endswith(".pdf")]

chunks_para_rag = preparar_chunks_detectando_tablas(
    textos_limpios,
    nombres_archivos,
    chunk_size=500,
    overlap=50)

len(chunks_para_rag)

6614

Guardamos los chunks_para_rag para evitar la ejecución de nuevo

In [8]:
import pickle
import os

# Se define la ruta del directorio
directory_path = "/content/drive/MyDrive/forma_dividida"

# Si dicha ruta no existe, se crea
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Se define el directorio del archivo
file_path = os.path.join(directory_path, "fase_1_chunks.pkl")

# Se guardan los chunks generados para evitar la ejecución de nuevo y evitar carga computacional
with open(file_path, "wb") as f:
    pickle.dump(chunks_para_rag, f)